# 训练模型测试

In [9]:
import os
from torchvision import transforms 
import torch.nn as nn
import torch
from tqdm import tqdm_notebook as tqdm
from Dataset import YouTubeBeDataset
from Network import NANnet

In [10]:
os.environ['CUDA_VISIBLE_DEVICES'] ='1' # 设置跑第几个GPU
# 使用cuda运算
device=torch.device("cuda")

In [11]:
model=NANnet(device=device)
model = model.train()
# if os.path.exists("model.acc.pth"):
#     model.load_state_dict(torch.load("model.acc.pth"))

### 初始化数据集

In [12]:
dataset = YouTubeBeDataset()
dataload = torch.utils.data.DataLoader(dataset, shuffle=True, batch_size=8, num_workers=2)

In [13]:
 model.load_state_dict(torch.load("model.acc.pth"))

# 训练

In [ ]:

# model.init_weights()
acc_max = 0
optimizer = torch.optim.Adadelta(model.parameters(),lr=0.05)
# optimizer=torch.optim.Adagrad(model.parameters(), lr=0.0005, lr_decay=0, weight_decay=0, initial_accumulator_value=0)
for epoch in range(300):
    total_loss = 0
    total_acc = 0
    total_size = 0
    bar = tqdm(dataload)
    for i,(x1,x2,y) in enumerate(bar):
        optimizer.zero_grad()
        x1, x2, y = x1.to(device), x2.to(device), y.to(device)
        r1, r2, acc, loss = model.process(x1, x2,y)
        total_size += x1.size(0)*x1.size(1)
        
        loss.backward()
        optimizer.step()
        # b=pred.item()
        total_acc += acc
        total_loss += loss.item()
        
        bar.set_postfix(loss=f"{total_loss/(i+1):0.4f}",
                       acc=f"{total_acc/total_size:0.4f}",
                       epoch=f"{epoch+1}")
    acc = total_acc/total_size
    torch.save(model.state_dict(),"model.acc.pth")
    if acc_max < acc:
        acc_max = max(total_acc/total_size,acc_max)
        torch.save(model.state_dict(),f"./checkpoints/model.acc{acc_max:0.4f}.pth")
#     if acc>0.8:
#         optimizer = torch.optim.Adadelta(model.parameters(),lr=1-acc)